In [1]:
%reload_ext autoreload
%autoreload 2

## 网络层

### Input层


- 参数
    
属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:----
<img width=150/> | <img width=100/> | <img width=200/> | <img width=450/>
in_features | [int, int, int] | [3, width, height] | 参数必须包含三个值; 非法时提示"in_features参数错误" <br>[0]: 通道数,一般固定位3; 非法时提示"in_features参数错误" <br> [1]: 用户设置的图片input_size的width; 非法时提示"in_features参数错误"<br> [2]: 用户设置的图片input_size的height; 非法时提示"in_features参数错误"

**Input层in_features可以设置成只读, 其值为(3, input_size.width, input_size.height), 从而减少用户输入错误** 

- 形状

```js
shape = {channel: this.in_features[0], width: this.in_features[1], height: this.in_features[2]}
```

- 实例

![Input Node](/notebooks/assets/layers/input_layer.png)

### Output层

- 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:----
<img width=150/> | <img width=100/> | <img width=200/> | <img width=450/>
out_features | int | > 0 | 值等于上一个层的out_features, 并且如果是分类任务其值应该等于分类的数目, 实际上这个层没有意义, 只用来作为显示

- 实例

![Output Node](/notebooks/assets/layers/output_layer.png)

### Convolution层 (2D)

- 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=100/> | <img width=450/>
in_channels | int | > 0 | 非法时提示"参数in_channels应为大于0的整数, 且值为上一层的shape输出中的out_channel" <br> 为了防止用户输入错误,可以改为只读形式,值为前一层的shape中channel
out_channels |  int | > 0 | 非法时提示"参数out_channels应为大于0的整数"
kernel_size | int | > 0 | 非法时提示"参数kernel_size应为大于0的整数 (推荐3,5,7等最大公约数为1的数)"
stride | int | > 0 | 非法时提示"参数stride应为大于0的整数"
padding | int | >=0 | 非法时提示"参数padding应为大于等于0的整数"
dilation | int | > 0 | 非法时提示"参数dilation应为大于0的整数"
groups | int | > 0 | 非法时提示"参数groups为大于0的整数"
bias | bool | |


**padding非0值不应大于kernel_size的一半, 如果dilation非0, padding不应大于`kernel_size + (kernel_size - 1)*(dilation - 1)`的一半**

- 形状

```js
    let w = prelayer.shape.w; // 前一层shape输出的宽(width)
    let h = prelayer.shape.h; // 前一层shape输出的高(height)
    let k = eval(`${this.kernel_size} + (${this.kernel_size} - 1) * (${this.dilation} - 1)`);  // 考虑到dilation大于1的情况

    wout = Math.floor(eval(`(${w} + 2 * ${this.padding} - ${k})/${this.stride}`)) + 1;
    hout = Math.floor(eval(`(${h} + 2 * ${this.padding} - ${k})/${this.stride}`)) + 1;
    
    shape = {channel: this.out_channels, width: wout, height: hout}
```

- 实例

当dilation == 1时: `k = kernel_size`

![Convoluation Node](/notebooks/assets/layers/convolution_layer.png)

当dilation > 1时, `k = kernel_size + (kernel_size -1)*(dilation -1)`

$$
\begin{align*}
shape.w &= (preinput.w + 2*padding - k) / stride + 1 \\
shape.h &= (preinput.h + 2*padding - k) / stride + 1
\end{align*}
$$


### Padding层(只考虑中心对称)

`零填充, 常数填充, 镜像填充, 重复填充`几种子类型方式可按一种方式处理, 只考虑对称方式的填充(top = bottom = left = right)

- 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=100/> | <img width=450/>
padding | int | >= 0 | 意义为(padding_left, padding_right, padding_top, padding_bottim)
value | float | >= 0 | 常数填充时, 需要填充的值

- 形状

```js
    let w = prelayer.shape.w + 2*padding
    let h = prelayer.shape.h + 2*padding
    let c = prelayer.shape.c
    
    shape = {channel: c, width: w, height: h}
```

- 实例

![Padding Node](/notebooks/assets/layers/padding_layer.png)

### Pooling层

- 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=100/> | <img width=450/>
kernel_size | int | > 0 | 非法时提示"参数kernel_size应为大于0的整数 (推荐3,5,7等最大公约数为1的数)"
stride | int | > 0 | 非法时提示"参数stride应为大于0的整数"
padding | int | >=0 | 非法时提示"参数padding应为大于等于0的整数"
dilation | int | > 0 | 非法时提示"参数dilation应为大于0的整数"
output_size | int | > 0 | (only AdaptiveXXXPool)自动适配池化的输出大小, 非法时提示"参数output_size应为大于0的整数"
ceil_mode | bool | True/False | True: 向上取整; False: 向下取整
~~return_indices~~ | bool | True/False | (only MaxPool)记录池化像素索引, 用来反池化, **可以先忽略/隐藏该参数**
~~count_include_pad~~ | bool | True/False | (only AvgPool)填充值是否参与计算, **可以先忽略/隐藏该参数**

- 形状

```js

    let w = prelayer.shape.w;   // 前一层shape的宽度
    let h = prelayer.shape.h;   // 前一层shape的高度
    let c = prelayer.shape.c;   // 前一层shape的通道
 
    // subtype: AdaptiveXXXPool2D 
    let W = output_size;
    let H = output_size;

    // subtype: 其他
    if (ceil_mode == "True") {
        func = Math.ceil;     // 上取整方法
    } else { 
        func = Match.floor;   // 下取整方法
    }
    let K = eval(`${kernel_size} + (${kernel_size} - 1) * (${dilation} - 1)`);  // 考虑到dilation大于1的情况
    let W = func(eval(`(${w} + 2 * ${padding} - ${K})/${stride}`)) + 1
    let H = func(eval(`(${h} + 2 * ${padding} - ${K})/${stride}`)) + 1
```

- 实例

XXXPool2D: `shape = {channel:prelayer.shape.c, w: func(w + 2*padding - k)/2) + 1, h: func(h + 2*padding - k)/2) + 1}`

![Pooling Node](/notebooks/assets/layers/pooling_layer.png)

AdaptiveXXXPool2D: `shape = {channel:prelayer.shape.c, w: output_size, h: output_size}`



### Activation层

- 形状

该层不会对shape做变化

```js    
    let w = prelayer.shape.w;
    let h = prelayer.shape.h;
    let c = prelayer.shape.c;
    
    shape = {channel: c, width: w, height: h}
```

- 实例


![Activation Node](/notebooks/assets/layers/activation_layer.png)

### Normalize层

$$
y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta
$$

- 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=100/> | <img width=450/>
num_features | int | > 0 | 与上一层的shape.c相等, 若非法提示"num_features特征数目不对"; **推荐设置成只读**
~~eps~~ | float | (0, 1) | 防止分母为0, 所以该属性**推荐隐藏** 
momentum | float | (0, 1) | 动态均值和动态方差所使用的动量因子(新batch和全局的取舍$\hat{x}_\text{new} = (1 - \text{momentum}) \times \hat{x} + \text{momentum} \times x_t$)
affine | bool | True/False | True: 对结果缩放($\gamma$)和平移($\beta$), 均可在反向传播中学习
track_running_stats | bool | Ture/False | True: 启动全局batch统计特性

**所有不在值域范围内的参数值均可提示参数非法**


- 形状

```js
    let w = prelayer.shape.w + 2*padding
    let h = prelayer.shape.h + 2*padding
    let c = prelayer.shape.c
    
    shape = {channel: c, width: w, height: h}
```

- 实例

![Normal Node](/notebooks/assets/layers/normalize_layer.png)

### Flatten层

将多维数据"压平"为1维

- 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=100/> | <img width=450/>
~~start_dim~~ | int | >= 0 | **删除属性**
~~end_dim~~ | int | >= -1 | **删除属性**

- 形状

```js
    let w = prelayer.shape.w;
    let h = prelayer.shape.h;
    let c = prelayer.shape.c;
    
    shape = {channel: null, width: null, height: w*h*c}
```

- 实例


![Flatten Node](/notebooks/assets/layers/flatten_layer.png)

### ~~Reshape层~~

**使用flatten层取代**

- 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=100/> | <img width=450/>
target_shape | list(int) | len(list) <= 3 | list里所有元素的乘积应该等于上一层shape的w*h*c <br> 非法提示"target_shape参数非法" 
    

- 形状
    
```js
    let c = prelayer.shape.c;
    let w = prelayer.shape.w;
    let h = prelayer.shape.h;

    let C = null;
    let W = null;
    let H = null;
    switch(len(target_shape)) {
        case 3:
            C = target_shape[0];
            W = target_shape[1];
            H = target_shape[2];
            break;
        case 2:
            W = target_shape[0];
            H = target_shape[1];
            break;
        case 1:
            H = target_shape[0];
            break;
        case *:
            // error
    }

    shape = {channel: C, width: W, height: H}

```

- 实例


![Reshape Node](/notebooks/assets/layers/reshape_layer.png)

### Linear层

- 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=100/> | <img width=450/>
in_features | int | > 0 | 值必须为上个层的shape.h, 非法提示"in_features参数非法"; **可以设置成只读**
out_features | int | > 0 |
bias | bool | True/False |

- 形状

```js
    shape = {channel: null, width:null, height:out_features}
```

- 实例


![Linear Node](/notebooks/assets/layers/linear_layer.png)

### Dropout层

- 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=100/> | <img width=450/>
p | float | (0, 1) | 伯努利(0-1)以p几率让神经元失效
inplace | bool | True/False | True: 对输入tensor本地修改, 节省内容 **推荐为默认**

- 形状

输入: 任意

输出: 输入

### Add层

- 形状

输入: 任意(两个input层维度一致,大小一致)

输出: 输入

### Cat层

- 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=100/> | <img width=450/>
dim | int | > 0 | -1: 表示最后一维度, 输入的层要大于2个, 除了要合并的dim外, 其他维度必须一致, 否则提示"dim参数错误"

- 形状

```js
   //            0  1   2   3
   // dim= 1:  (32, 3, 64, 64), (32, 3, 64, 64) --> (32, 3+3, 64, 64)
   // dim=-1:  (32, 3, 64, 64), (32, 3, 64, 64) --> (32, 3, 64, 64 + 64)

```

- 实例


![Cat Node](/notebooks/assets/layers/cat_layer.png)

### ~~Loss层~~

**删除或者隐藏**

In [2]:
import torch

A = torch.ones(2, 3, 2)
A, A.shape

(tensor([[[1., 1.],
          [1., 1.],
          [1., 1.]],
 
         [[1., 1.],
          [1., 1.],
          [1., 1.]]]),
 torch.Size([2, 3, 2]))

In [3]:
B = 2*torch.ones(2, 3, 2)
B, B.shape

(tensor([[[2., 2.],
          [2., 2.],
          [2., 2.]],
 
         [[2., 2.],
          [2., 2.],
          [2., 2.]]]),
 torch.Size([2, 3, 2]))

In [4]:
C = torch.cat((A,B), 2)
C, C.shape

(tensor([[[1., 1., 2., 2.],
          [1., 1., 2., 2.],
          [1., 1., 2., 2.]],
 
         [[1., 1., 2., 2.],
          [1., 1., 2., 2.],
          [1., 1., 2., 2.]]]),
 torch.Size([2, 3, 4]))

# <div align='center'> 拖拽模型 </div>

In [5]:
from k12libs.utils.nb_easy import k12ai_start_html, W3URL
k12ai_start_html(f'{W3URL}/drawnet.html?jfile=simple', height=780, flask=True)

 * Serving Flask app "k12libs.utils.nb_easy" (lazy loading)


 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Input/Output Layers

###  Input

### Output

## Core Layers

###  Activation

###  ActivityRegularization

### Dense

### Dropout

### Flatten

### Highway

### Lambda

### Masking

### MaxoutDense

### Permute

### RepeatVector

### Reshape

### SpatialDropout1D

### SpatialDropout2D

### SpatialDropout3D

### TimeDistributedDense

## Convolutional Layers

### AtrousConvolution1D

### AtrousConvolution2D

### Convolution1D

### Convolution2D

### Convolution3D

### Cropping1D

### Cropping2D

### Cropping3D

### Deconvolution2D

### SeparableConvolution2D

### UpSampling1D

### UpSampling2D

### UpSampling3D

### ZeroPadding1D

### ZeroPadding2D

### ZeroPadding3D

## Pooling Layers

### AvgPooling1D

### AvgPooling2D

### AvgPooling3D

### MaxPooling1D

### MaxPooling2D

### MaxPooling3D

### GlobalAvgPooling1D

### GlobalAvgPooling2D

### GlobalAvgPooling3D

### GlobalMaxPooling1D

### GlobalMaxPooling2D

### GlobalMaxPooling3D

## Local Layers

### LocallyConnected1D

### LocallyConnected2D

## Recurrent Layers

### GRU

### LSTM

### SimpleRNN

## Embeddings Layers

### Embeding

## Convolutional Recurrent Layers

### ConvLSTM2D

### ConvRecurrent2D

## Advanced Activations Layers

### ELU

### LeakyReLU

### ParametricSoftplus

### PReLU

### SReLU

### ThreasholdedReLU

## Normalization Layers

### BatchNormalization

## Noise Layers

### GaussianDropout

### GaussianNoise

## Special Functions

### Merge

## Pre-Trained Models

### InceptionV3

### ResNet50

### SqueezeNet

### VGG16

### VGG19

### WideResNet

## References

### [deepcognition](https://app.deepcognition.ai)